# Problem Statement 

Suppose we are playing a game, let's play a simple coin slot machine, the ones like we used to play as kids on the beach. For now we only have one slot
in the machine where we can drop our penny. We drop our penny and we will recieve some reward, in our world this is a numerical value. For instance we have dropped our
penny such that we have knocked another three pennies off, thus the reward for that action can be formalised as +3. Our objective is to come out of the slot machine
room with more pennies than we started with, ideally holding every penny that we could possibly win within a set timeframe. 

Our penny machines normall have more than one slot, we can extend our problem to whats known as k-armed Bandits. We have k slots, where the action we take is which slot to play
and the reward is how many pennies did we win on that action.

Each action within our bandit as an expected reward for that given selected action, this is sometimes known as the action-value. Where value is another word for our reward. 
At some timestep $t$, we selected an action $A_t$, and recieve some numerical reward $R_t$. For an arbitary action our action-value can be stated as:

$$
q(a) = \mathbb{E}[R_t | A_t = a]
$$

One can see that if we known all the values for every action, the solution is as simple as choosing the action with the highest value.
We do not know the exact action-values, instead we have an estimate $Q_t(a)$.


# Living life as a Bandit

Within our algorithm we keep track of our action-value estimates $Q_t(a)$, infact this what we are trying to solve. At every step we have a action value that is greater than another.
If you as a player have been consistently adding coins to the right hand side of the slot machine you probably have a higher chance of winning coins on the right hand side of the machine and so
the action-values will correspond to that notion. Choosing an action from these slots on the right hand side of the machine is **exploiting** your underlying knowledge of the world.

Now your friend turns up and annoys you by placing a coin on the left hand side of the machine. This goes against your internal action-value map that you have. These are not the current highest
estimates. What your friend has just done is **explore** the world. For those who are reading who come from a machine learning world, this is the same concept as a gradient descent.
We have a world with potentially many avenues, all giving potentially different rewards. We then have a **conflict** between how much do we want to explore this world and recieve maybe some low
values, or how long do we want to keep **exploiting** the avenue we are currently in knowing we have a good reward (alas potentially not the best).

We shall take a look at some different algorithms which way up different versions of this conflict problem and show some nice results.


# Algorithm Formalisation

## Incremental action-value

We now focus on a single action. How can we calculate the action-value after $n$ rewards. Well this is simply the estimate after it has selection $n - 1$ rewards.

$$
Q_n = \frac{R_1 + R_2 + ... + R_{n-1}}{n-1}
$$

However for this we will need to store all the previous rewards. How can we do it incrementally? Well the solution is to store a running sun and count.

$$
Q_{n+1} = Q_n + \frac{1}{n} [R_n - Q_n]
$$

I'm not going to show the proof within here but I would urge the reader to go and read a great book by Sutton and Barto, Reinforcement Learning an Introduction.

## Algorithm

The base algorithm can be seen in the pseudocode for incremental updates. Note, this is the algorithm for an epsilon greedy bandit.


## Random Bandit

The first bandit we will describe is a simple random bandit. This bandit has no care about what previous actions have been taken, nor what previous rewards have been achieved.
The random bandit algorithm Python code can be seen below.

In [ ]:
class Random(object):
  """A random agent.

  This agent returns an action between 0 and 'number_of_arms', uniformly at
  random. The 'previous_action' argument of 'step' is ignored.
  """

  def __init__(self, name, number_of_arms):
    """Initialise the agent.
    
    Sets the name to `random`, and stores the number of arms. (In multi-armed
    bandits `arm` is just another word for `action`.)
    """
    self._number_of_arms = number_of_arms
    self.name = name

  def step(self, unused_previous_action, unused_reward):
    """Returns a random action.
    
    The inputs are ignored, but this function still requires an action and a
    reward, to have the same interface as other agents who may use these inputs
    to learn.
    """
    return np.random.randint(self._number_of_arms)

  def reset(self):
    pass

## Epsilon Greedy

The epsilon greedy algorithm allows the bandit to balance the exploration vs exploitation conflict that we mentioned earlier. We introduce the parameter $\epsilon$ which is the probability
of choosing whether to explore, or exploit. If the proability 